In [1]:
import breeze.linalg.DenseVector
import io.github.mandar2812.dynaml.graphics.charts.Highcharts._
import io.github.mandar2812.dynaml.DynaMLPipe._
import io.github.mandar2812.dynaml.pipes._
import io.github.mandar2812.dynaml.utils.GaussianScaler
import scala.collection.mutable.{MutableList => ML}

import breeze.linalg.DenseVector

import io.github.mandar2812.dynaml.graphics.charts.Highcharts._

import io.github.mandar2812.dynaml.DynaMLPipe._

import io.github.mandar2812.dynaml.pipes._

import io.github.mandar2812.dynaml.utils.GaussianScaler

import scala.collection.mutable.{MutableList => ML}

In [2]:
val deltaOperationARXMultiOutput = (deltaT: List[Int], deltaTargets: List[Int]) =>
    DataPipe((lines: Iterable[(Double, DenseVector[Double])]) =>
      lines.toList.sliding(math.max(deltaT.max, deltaTargets.max) + 1).map((history) => {

        val hist = history.take(history.length - 1).map(_._2)

        val num_outputs = deltaTargets.length

        val featuresAcc: ML[Double] = ML()

        val lags = deltaT ++ deltaTargets

        (0 until hist.head.length).foreach((dimension) => {
          //for each dimension/regressor take points t to t-order
          featuresAcc ++= hist.takeRight(lags(dimension))
            .map(vec => vec(dimension))
        })

        val outputs = history.last._2(0 until num_outputs)
        val features = DenseVector(featuresAcc.toArray)

        (features, outputs)
      }).toStream)

deltaOperationARXMultiOutput: (List[Int], List[Int]) => DataPipe[Iterable[(Double, DenseVector[Double])], Stream[(DenseVector[Double], DenseVector[Double])]] = ammonite.$sess.cmd1$Helper$$Lambda$2130/0x0000000840af3040@411aba43

In [3]:
val deltaT = 2

deltaT: Int = 2

In [4]:
val preProcessPipe = fileToStream >
  trimLines >
  replaceWhiteSpaces >
  extractTrainingFeatures(
    List(0,5,6,7,8,1,2,3,4),
    Map()
  ) >
  removeMissingLines >
  IterableDataPipe((line: String) => {
    val splits = line.split(",")
    val timestamp = splits.head.toDouble
    val data_vec = DenseVector(splits.tail.map(_.toDouble))
    (timestamp, data_vec)
  }) >
  deltaOperationARXMultiOutput(
    List.fill(4)(deltaT),
    List.fill(4)(deltaT))

preProcessPipe: ComposedPipe[String, Iterable[(Double, DenseVector[Double])], Stream[(DenseVector[Double], DenseVector[Double])]] = ComposedPipe(
  ComposedPipe(
    ComposedPipe(
      ComposedPipe(
        ComposedPipe(
          ComposedPipe(
            io.github.mandar2812.dynaml.pipes.DataPipe$$anon$2@56c77273,
            io.github.mandar2812.dynaml.pipes.IterableDataPipe$$anon$2@3a1fc26c
          ),
          io.github.mandar2812.dynaml.pipes.IterableDataPipe$$anon$2@6c1bacc1
        ),
        io.github.mandar2812.dynaml.pipes.DataPipe$$anon$2@4d10e0ee
      ),
      io.github.mandar2812.dynaml.pipes.IterableDataPipe$$anon$4@4f5ea719
    ),
    io.github.mandar2812.dynaml.pipes.IterableDataPipe$$anon$2@b16103c
  ),
  io.github.mandar2812.dynaml.pipes.DataPipe$$anon$2@3adad83b
)

In [5]:
preProcessPipe("../data/steamgen.csv")

res4: Stream[(DenseVector[Double], DenseVector[Double])] = Stream(
  (
    DenseVector(320.08238893611104, 321.7109901970145, 2.5067738530481467, 2.5459079477298627, 0.032701299739161395, 0.2847994364405881, 9.302970053314816, 9.66262058403806, 0.7235675006200512, 0.5271473870728786, 0.6916286704298222, 0.3830201937658643, -2.28305641205371, -3.716481387850976, 0.017982753309452384, 0.018717890028551235),
    DenseVector(320.91330769424036, 2.3605615428942226, 0.20365164556689003, 10.990955477930623)
  ),
  (
    DenseVector(321.7109901970145, 320.91330769424036, 2.5459079477298627, 2.3605615428942226, 0.2847994364405881, 0.20365164556689003, 9.66262058403806, 10.990955477930623, 0.5271473870728786, 0.589603122994395, 0.3830201937658643, 0.7058146274340293, -3.716481387850976, -1.5309929715976218, 0.018717890028551235, 0.020756328419717966),
    DenseVector(325.0025177260005, 0.02705418231106365, 0.3261868791391608, 12.430107125754343)
  ),
  (
    DenseVector(320.91330769424036, 325.0